In [4]:
from polygon_tally import *
import threading

#from numba.experimental import jitclass
#from numba import jit

In [6]:
path = "output/different-polygon-function/"

def write_output(string, file_path):
    f = open(file_path, 'a')
    f.write(string + '\n')
    f.close()

# parameters
order = 35
radius = 1
mesh_size = 5_000
side_num = 5

def function(x, y):
    return x**2 + y**2


# creating the save path
loc_path = path + "{}sides/".format(side_num)
"""
if not os.path.isdir(loc_path):
    os.mkdir(loc_path)
    os.mkdir(loc_path + "z_basis/")
    os.mkdir(loc_path + "k_basis/")
"""

def tout(label):
    tf = time.time()
    print(label, tf - t0)

t0 = time.time()
# getting approximation paths
z1 = ZApprox(side_num, radius, mesh_size)
tout("z approx")
k1 = KApprox(side_num, radius, mesh_size)
tout("k approx")
x, y = z1.x, z1.y
tout("assigning variables")


print("starting {} sided polygon".format(side_num))
# analytical
Fa = function(x, y)
bounds = [np.min(Fa), np.max(Fa)]
tout("analytical")

Fa_l2_norm = la.norm(Fa, 2)
Fa_linf_norm = la.norm(Fa, np.inf)

# approx
Fz = 0
Fk = 0


# plotting analytical
#k1.plotter(Fa, bounds, "Analytical", loc_path + "ana.png")


# calculating the coefficients
for n in range (order + 1):
    print("--- n ---", n)
    for m in np.arange(-n, n + 1, 2):
        print("m", m)
        # z basis
        bz = ZBasis(n, m, side_num, radius)
        bz._gen_all_cd(mesh_size)
        tout("z basis")
        
        cz = bz.num_cz_nm(mesh_size)
        cz_out = (str(n) + "," + str(m) + "," + str(cz))
#        write_output(cz_out, loc_path + "czs.txt")
        tout("cz")
        
        Fz += np.float64(cz * bz.z_nm(x, y))

        # k basis
        bk = KBasis(n, m, side_num, radius)
        bk._gen_all_cd(mesh_size)
        tout("k basis")
        
        ck = bk.num_ck_nm(mesh_size)
        ck_out = (str(n) + "," + str(m) + "," + str(ck))
#        write_output(ck_out, loc_path + "cks.txt")
        tout("ck")
        
        Fk += np.float64(ck * bk.k_nm(x, y))

    print("norms")
    # norm calcs
    Dz = Fa - Fz
    Dk = Fa - Fk
    tout("diffs")
    
    l2_z = la.norm(Dz, 2) / Fa_l2_norm
    l2_k = la.norm(Dk, 2) / Fa_l2_norm
    tout("l2s")
    
    linf_z = la.norm(Dz, np.inf) / Fa_linf_norm
    linf_k = la.norm(Dk, np.inf) / Fa_linf_norm
    tout("linfs")

z approx 1.8207805156707764
k approx 4.301883935928345
assigning variables 4.320014715194702
starting 5 sided polygon
analytical 4.569368362426758
--- n --- 0
m 0
z basis 14.22432827949524
cz 18.056692361831665
k basis 22.6523756980896
ck 41.63337969779968
norms
diffs 43.72275352478027
l2s 59.31062436103821
linfs 59.440536975860596
--- n --- 1
m -1
z basis 61.54268479347229
cz 66.42909669876099
k basis 71.0590398311615


KeyboardInterrupt: 

In [ ]:
    print("text out")
    # writing text outputs
    write_output(str(l2_z), loc_path + "l2z.txt")
    write_output(str(l2_k), loc_path + "l2k.txt")
    write_output(str(linf_z), loc_path + "linfz.txt")
    write_output(str(linf_k), loc_path + "linfk.txt")

    print("imgs out")
    # img outputs
    z1.plotter(Fz, bounds, "Z Order = {}".format(n), loc_path + "z_basis/z{}.png".format(n))
    k1.plotter(Fk, bounds, "K Order = {}".format(n), loc_path + "k_basis/k{}.png".format(n))
